## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-16-52-32 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/11/18/world/...
1,2025-11-18-16-50-22 +0000,bbc,Brazil creates new Indigenous territories duri...,https://www.bbc.com/news/articles/c1d0vekq12ro...
2,2025-11-18-16-49-58 +0000,nyt,"Live Updates: Trump Welcomes Saudi Leader, Wit...",https://www.nytimes.com/live/2025/11/18/world/...
3,2025-11-18-16-47-26 +0000,nypost,Chuck Schumer is the most unpopular he’s ever ...,https://nypost.com/2025/11/18/us-news/chuck-sc...
4,2025-11-18-16-45-26 +0000,nyt,"Cloudflare Outage Disrupts X, ChatGPT and Othe...",https://www.nytimes.com/2025/11/18/business/cl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,51
182,epstein,19
4,new,15
231,files,12
167,house,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
277,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...,130
37,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,125
212,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,109
215,2025-11-18-01-48-49 +0000,nyt,Trump Has the Power to Release the Epstein Fil...,https://www.nytimes.com/2025/11/17/us/politics...,106
106,2025-11-18-12-38-40 +0000,bbc,When is the Epstein files vote - and why does ...,https://www.bbc.com/news/articles/cq50nvq8znvo...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
277,130,2025-11-17-21-29-00 +0000,wsj,Trump said he would sign legislation to releas...,https://www.wsj.com/politics/policy/trump-says...
89,56,2025-11-18-13-24-46 +0000,cbc,"Fighter jet, investment deals on agenda as Sau...",https://www.cbc.ca/news/world/washington-saudi...
276,48,2025-11-17-21-29-00 +0000,wsj,New York Officials to Team Up With Wall Street...,https://www.wsj.com/us-news/new-york-officials...
174,40,2025-11-18-09-18-41 +0000,wapo,U.N. Security Council approves Trump’s 20-poin...,https://www.washingtonpost.com/national-securi...
294,39,2025-11-17-20-22-42 +0000,nypost,Ex-Disney star ripped for ‘demonic’ app that l...,https://nypost.com/2025/11/17/business/ex-disn...
37,32,2025-11-18-15-49-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
9,32,2025-11-18-16-35-40 +0000,bbc,Minister insists government committed to asylu...,https://www.bbc.com/news/articles/cvgd2vq4enzo...
195,31,2025-11-18-04-02-54 +0000,cbc,Ex-Harvard president Larry Summers 'stepping b...,https://www.cbc.ca/news/world/summers-epstein-...
127,31,2025-11-18-11-32-02 +0000,nypost,Cheerleader who died on Carnival cruise was fo...,https://nypost.com/2025/11/18/us-news/cheerlea...
280,30,2025-11-17-21-22-00 +0000,wsj,Fed Vice Chair Philip Jefferson acknowledged t...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
